In [1]:
import re


def process_law_documents(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        # 파일 전체를 읽은 후 3개의 줄바꿈을 기준으로 나눕니다.
        raw_docs = re.split(r'\n\n\n', f.read())
    
    cleaned_docs = []

    for doc in raw_docs:
        doc = doc.strip()
        if not doc:
            continue
        
        # 1. 상단 불필요한 메뉴 제거
        # 메뉴 패턴이 포함되어 있다면 그 이후 내용만 추출
        if "한눈보기" in doc:
            parts = doc.split("한눈보기", 1)
            doc = parts[1].strip()
        
        # 2. 하단 부칙 및 그 이후 내용 제거
        # '부 칙'이라는 글자가 나오면 그 앞까지만 사용
        body_parts = re.split(r'\n\s*부\s*칙', doc)
        doc_body = body_parts[0].strip()

        if doc_body:
            cleaned_docs.append(doc_body)
            
    return cleaned_docs

file_path = "..\B-TEAM\사회복지_법령_전체.txt" 
final_law_list = process_law_documents(file_path)

# print(f"추출된 문서 개수: {len(final_law_list)}개")
# if final_law_list:
#     print("-" * 30)
#     print("추출된 첫 번째 문서 본문 맛보기:")
#     print(final_law_list[0][:150] + "...")

<>:32: SyntaxWarning: invalid escape sequence '\B'
<>:32: SyntaxWarning: invalid escape sequence '\B'
C:\Users\USER\AppData\Local\Temp\ipykernel_26584\2600586122.py:32: SyntaxWarning: invalid escape sequence '\B'
  file_path = "..\B-TEAM\사회복지_법령_전체.txt"


In [30]:
print(final_law_list[-1][:150] + "...")

성평등가족부 직제 시행규칙
[시행 2026. 1. 6.] [성평등가족부령 제4호, 2026. 1. 6., 일부개정]
성평등가족부(혁신행정법무담당관), 02-2100-6082
        제1장 총칙
제1조(목적) 이 규칙은 성평등가족부에 두는 보조기관ㆍ보좌기관의 직급...


In [2]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from langchain_core.documents import Document
from qdrant_client.http.models import Distance, VectorParams
from langchain_core.vectorstores import InMemoryVectorStore
from dotenv import load_dotenv

load_dotenv()

c:\project\3rd\SKN21-3rd-1TEAM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
# 2. 임베딩 및 벡터 DB 설정
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("woong0322/ko-legal-sbert-finetuned")

embeddings = model.encode(final_law_list)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [3, 3]

c:\project\3rd\SKN21-3rd-1TEAM\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--woong0322--ko-legal-sbert-finetuned. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to r

torch.Size([243, 243])


In [ ]:
vectorstore = InMemoryVectorStore.from_documents(
    documents=final_law_list,
    embedding=embeddings
)